# Digital Tools for Finance - Project
**Research Question:**\
What is the effect of the U.S. GDP growth rate on different S&P500 sector indices' market capitalizations\

**Authors:**\
Jonas Neller, 21-730-676\
Lorena Tassone, 18-700-237\
Naomi Huser, 17-056-201

## ToDo List:
API einrichten mit FRED\
Up- and downstream functions for database\
<s>Data cleaning</s> -> NaN values?\
Regressions\
plots

create docker container for jupyter-notebook and app

overleaf doc\
overleaf presentation

--> interactive app\
--> jupyter notebook \
--> good comments in code and github commits!


Notes:
- Fore presentation use beamer!
- Use ipywidget for app of Jupyter Notebook!

## Data Cleaning

In [1]:
# libraries
import numpy as np
import pandas as pd
import os
from functools import reduce


In [2]:
# create dataframes
data_mcap = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Market_Cap')
data_gdp = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Real_GDP').set_index('Date ')
data_unempr = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Unemployment_rate').set_index('Date ')
data_crudeprice = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Crude_price').set_index('Date ')
data_feddebt = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Federal_debt').set_index('Date ')
data_fedrate = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Federal_rate').set_index('Date ')

data_mcap = data_mcap.transpose()
data_mcap.columns = data_mcap.loc['Sector',:]


# create sector subsets
sectors = data_mcap.loc['Sector',:].unique()
print(sectors)

communication = data_mcap[['Communication Services']]  
consumerdisc = data_mcap[['Consumer Discretionary']] 
consumerstap = data_mcap[['Consumer Staples']] 
energy = data_mcap[['Energy']] 
financials = data_mcap[['Financials']] 
healthcare = data_mcap[['Health Care']] 
industrials = data_mcap[['Industrials']] 
informationtech = data_mcap[['Information Technology']] 
materials = data_mcap[['Materials']] 
realestate = data_mcap[['Real Estate']] 
utilities = data_mcap[['Utilities']] 

utilities

['Communication Services' 'Consumer Discretionary' 'Consumer Staples'
 'Energy' 'Financials' 'Health Care' 'Industrials'
 'Information Technology' 'Materials' 'Real Estate' 'Utilities']


Sector,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,...,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities
Name,AES CORP,ALLIANT ENERGY CORP,AMEREN CORPORATION,AMERICAN ELECTRIC POWER,AMERICAN WATER WORKS CO INC,ATMOS ENERGY CORP,CENTERPOINT ENERGY INC,CMS ENERGY CORP,CONSOLIDATED EDISON INC,CONSTELLATION ENERGY,...,NISOURCE INC,NRG ENERGY INC,P G & E CORP,PINNACLE WEST CAPITAL,PPL CORP,PUBLIC SERVICE ENTERPRISE GP,SEMPRA ENERGY,SOUTHERN CO/THE,WEC ENERGY GROUP INC,XCEL ENERGY INC
ISIN,US00130H1059,US0188021085,US0236081024,US0255371017,US0304201033,US0495601058,US15189T1079,US1258961002,US2091151041,US21037T1097,...,US65473P1057,US6293775085,US69331C1080,US7234841010,US69351T1060,US7445731067,US8168511090,US8425871071,US92939U1060,US98389B1008
Sector,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,...,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities
2006-12-29 00:00:00,14638712578.440001,4384709584.45254,11082465319.5,16843471727.5,NaN,2783812955.56,5186872195.1,3714659289.6,12344091521.74,NaN,...,2585436457.09001,7082069181.42,16509186008.719999,5042522737.47,12668866168.236,16741258572.139999,14692202603.76,27377404804.080002,5551817787.18,9382708997.219999
2007-03-30 00:00:00,14293334604.719999,5203142483.98959,10374986145.0,19337893380.0,NaN,2770689248.16,5742217475.28,3973954379.2,13155459853.74,NaN,...,2630056060.93063,8813046970.639999,16933949864.25,4799797239.75,14457495295.370501,20942966433.119999,16040666104.379999,27435978163.0,5674427245.96,10067390534.67
2007-06-29 00:00:00,14619747943.040001,4510086514.57497,10170069510.9,17960461536.32,NaN,2669515424.1,5581703213.4,3840000860.8,12205768099.200001,NaN,...,2230398860.43326,10073112452.18,15922900532.4,3964184870.55,16710462345.6555,22263327147.599998,15625445214.120001,25779517271.189999,5172710574.79,8370499773.25
2007-09-28 00:00:00,13399013097.120001,4448559078.15501,10899085680.0,18395319997.439999,NaN,2525014003.68,5148532071.2,3777497632.16,12546154676.9,NaN,...,2062030361.8079,10142537781.450001,16890508337.0,3963216017.88,16457065022.064199,22374390646.650002,15345081970.639999,27443191550.279999,5266183705.17,9044078162.52
2007-12-31 00:00:00,14315413925.07,4723691869.76736,11276176509.389999,18643867338.240002,NaN,2516583130.2,5503085216.85,3903264497.44,13237141597.549999,NaN,...,2035175796.14,10343243513.459999,15256086158.67,4257329376.76,17988112199.596802,24983726704.959999,16176116207.16,29429803538.75,5696297037.12,9477821860.459999
2008-03-31 00:00:00,11189927437.98,3866971305.3726,9197420441.040001,16669763687.52,NaN,2288618752.5,4684342634.52,3048897541.34,10803922468.5,NaN,...,1857445107.65951,9218884263.26,13123393899.559999,3521507062.88,15877949067.0611,20441693328.02,13922387942.4,27231399981.990002,5143660652.47,8561502181.05
2008-06-30 00:00:00,12905493651.459999,3783440279.20662,8819415649.98,16156006131.15,3548800000.0,2486498185.65,5268654469.8,3355138357.9,10657320170.49,NaN,...,1930919693.56965,10121052813.299999,14179609590.93,3088847557.72,18091052073.992401,23361208622.939999,14131787158.6,26894940376.32,5287119732.02,8647303241.34


In [3]:
# merge factors by quarterly date
data_frames = [data_gdp, data_unempr, data_crudeprice, data_feddebt, data_fedrate]
data_factors_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date '], how='inner'), data_frames)
data_factors_merged

,Real_GDP,Delta_real_GDP,Unemployment_rate,Crude_price,Federal_debt,Delta_Fed_Debt,Federal_rate
Date,,,,,,,
2007-01-01,15478.956,NaN,4.6,54.780000,8849665,NaN,5.25
2007-04-01,15577.779,1.006384,4.5,63.974762,8867677,1.002035,5.25
2007-07-01,15671.605,1.006023,4.7,74.046364,9007653,1.015785,5.26
2007-10-01,15767.146,1.006096,4.7,85.926522,9229172,1.024592,4.76
2008-01-01,15702.906,0.995926,5.0,92.982609,9437594,1.022583,3.94
2008-04-01,15792.773,1.005723,5.0,112.336364,9492006,1.005765,2.28
2008-07-01,15709.562,0.994731,5.8,133.958261,10024725,1.056123,2.01
2008-10-01,15366.607,0.978169,6.5,76.283913,10699805,1.067341,0.97
2009-01-01,15187.475,0.988343,7.8,41.630909,11126941,1.039920,0.15


In [4]:
# checking timeframe length
print(len(utilities.index)-3)
print(len(data_factors_merged))

57
57


In [5]:
#Drop Companies with missing Market Cap Data
communication = communication.dropna(axis=1)
communication = communication.tail(-3)
communication.columns = np.arange(len(communication.columns))
communication = communication.set_index(data_factors_merged.index)


consumerdisc = consumerdisc.dropna(axis=1)
consumerdisc = consumerdisc.tail(-3)
consumerdisc.columns = np.arange(len(consumerdisc.columns))
consumerdisc = consumerdisc.set_index(data_factors_merged.index)

consumerstap = consumerstap.dropna(axis=1)
consumerstap = consumerstap.tail(-3)
consumerstap.columns = np.arange(len(consumerstap.columns))
consumerstap = consumerstap.set_index(data_factors_merged.index)

energy = energy.dropna(axis=1)
energy = energy.tail(-3)
energy.columns = np.arange(len(energy.columns))
energy = energy.set_index(data_factors_merged.index)

financials = financials.dropna(axis=1)
financials = financials.tail(-3)
financials.columns = np.arange(len(financials.columns))
financials = financials.set_index(data_factors_merged.index)

healthcare = healthcare.dropna(axis=1)
healthcare = healthcare.tail(-3)
healthcare.columns = np.arange(len(healthcare.columns))
healthcare = healthcare.set_index(data_factors_merged.index)

industrials = industrials.dropna(axis=1)
industrials = industrials.tail(-3)
industrials.columns = np.arange(len(industrials.columns))
industrials = industrials.set_index(data_factors_merged.index)

informationtech = informationtech.dropna(axis=1)
informationtech = informationtech.tail(-3)
informationtech.columns = np.arange(len(informationtech.columns))
informationtech = informationtech.set_index(data_factors_merged.index)

materials = materials.dropna(axis=1)
materials = materials.tail(-3)
materials.columns = np.arange(len(materials.columns))
materials = materials.set_index(data_factors_merged.index)

realestate = realestate.dropna(axis=1)
realestate = realestate.tail(-3)
realestate.columns = np.arange(len(realestate.columns))
realestate = realestate.set_index(data_factors_merged.index)

utilities = utilities.dropna(axis=1)
utilities = utilities.tail(-3)
utilities.columns = np.arange(len(utilities.columns))
utilities = utilities.set_index(data_factors_merged.index)

In [6]:
# take average mcap of each sector
data_factors_merged['mean_communication'] = communication.mean(axis=1)
data_factors_merged['mean_consumerdisc'] = consumerdisc.mean(axis=1)
data_factors_merged['mean_consumerstap'] = consumerstap.mean(axis=1)
data_factors_merged['mean_energy'] = energy.mean(axis=1)
data_factors_merged['mean_financials'] = financials.mean(axis=1)
data_factors_merged['mean_healthcare'] = healthcare.mean(axis=1)
data_factors_merged['mean_industrials'] = industrials.mean(axis=1)
data_factors_merged['mean_informationtech'] = informationtech.mean(axis=1)
data_factors_merged['mean_materials'] = materials.mean(axis=1)
data_factors_merged['mean_realestate'] = realestate.mean(axis=1)
data_factors_merged['mean_utilities'] = utilities.mean(axis=1)
data_factors_merged



,Real_GDP,Delta_real_GDP,Unemployment_rate,Crude_price,Federal_debt,Delta_Fed_Debt,Federal_rate,mean_communication,mean_consumerdisc,mean_consumerstap,mean_energy,mean_financials,mean_healthcare,mean_industrials,mean_informationtech,mean_materials,mean_realestate,mean_utilities
Date,,,,,,,,,,,,,,,,,,
2007-01-01,15478.956,NaN,4.6,54.780000,8849665,NaN,5.25,3.356684e+10,1.397676e+10,3.511489e+10,5.631075e+10,3.777613e+10,2.213235e+10,2.034991e+10,2.714489e+10,7.853161e+09,7.264137e+09,1.288313e+10
2007-04-01,15577.779,1.006384,4.5,63.974762,8867677,1.002035,5.25,3.830225e+10,1.450353e+10,3.521286e+10,5.643889e+10,3.701597e+10,2.241301e+10,2.038227e+10,2.642156e+10,9.070797e+09,7.742640e+09,1.391123e+10
2007-07-01,15671.605,1.006023,4.7,74.046364,9007653,1.015785,5.26,4.062438e+10,1.466297e+10,3.558817e+10,6.331786e+10,3.752751e+10,2.310317e+10,2.218927e+10,2.939174e+10,9.796573e+09,7.148311e+09,1.363948e+10
2007-10-01,15767.146,1.006096,4.7,85.926522,9229172,1.024592,4.76,4.162330e+10,1.503752e+10,3.673282e+10,6.927823e+10,3.745375e+10,2.337513e+10,2.335841e+10,3.130621e+10,1.056654e+10,7.338629e+09,1.379857e+10
2008-01-01,15702.906,0.995926,5.0,92.982609,9437594,1.022583,3.94,4.209691e+10,1.328580e+10,3.780151e+10,7.056624e+10,3.500410e+10,2.335016e+10,2.184990e+10,3.163154e+10,1.117767e+10,6.721945e+09,1.445557e+10
2008-04-01,15792.773,1.005723,5.0,112.336364,9492006,1.005765,2.28,3.465053e+10,1.201863e+10,3.712802e+10,6.354183e+10,3.104005e+10,2.034714e+10,2.069336e+10,2.665176e+10,1.075743e+10,6.579863e+09,1.285859e+10
2008-07-01,15709.562,0.994731,5.8,133.958261,10024725,1.056123,2.01,3.467687e+10,1.056875e+10,3.448510e+10,7.180266e+10,2.635974e+10,1.964691e+10,1.836990e+10,2.726984e+10,1.229384e+10,6.345413e+09,1.374376e+10
2008-10-01,15366.607,0.978169,6.5,76.283913,10699805,1.067341,0.97,3.012416e+10,1.073620e+10,3.633570e+10,5.642145e+10,2.907454e+10,1.945900e+10,1.669078e+10,2.410856e+10,8.464278e+09,6.663311e+09,1.161395e+10
2009-01-01,15187.475,0.988343,7.8,41.630909,11126941,1.039920,0.15,2.566946e+10,8.256185e+09,3.145109e+10,4.730873e+10,1.978422e+10,1.679154e+10,1.257121e+10,1.790199e+10,6.054703e+09,4.424134e+09,1.031568e+10


## Regression

In [90]:
# Regress mean market cap on GDP growth and controlling variables for all sectors

import statsmodels.regression.linear_model as sm
#import statsmodels.formula.api as sm => Spell ols small then, nicer summary table imo


list_of_sectors = ["mean_communication","mean_consumerdisc","mean_consumerstap","mean_energy","mean_financials","mean_healthcare","mean_industrials","mean_informationtech","mean_materials","mean_realestate","mean_utilities"]

# list of models
models = []

for resp in list_of_sectors:
    formula = resp + " ~ Real_GDP + Unemployment_rate + Crude_price + Delta_Fed_Debt + Federal_rate"
    models.append(sm.OLS.from_formula(formula, data = data_factors_merged).fit())

# Summarize regression results
models[0].summary()




<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     mean_communication   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.963
Method:                 Least Squares   F-statistic:                     289.3
Date:                Mon, 05 Dec 2022   Prob (F-statistic):           1.21e-35
Time:                        00:19:41   Log-Likelihood:                -1369.1
No. Observations:                  56   AIC:                             2750.
Df Residuals:                      50   BIC:                             2762.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -8.157e+11   9.01e+10     -9.054      0.000   -9.97e+11   -6.35e+11
Real_GDP           4.329e+07   1.69e+06     25.656      0.000    3.99e+07    4.67e+07
Unemployment_rate  3.348e+09   1.14e+09      2.945      0.005    1.06e+09    5.63e+09
Crude_price       -2.236e+08   7.29e+07     -3.067      0.003    -3.7e+08   -7.72e+07
Delta_Fed_Debt     1.539e+11   8.54e+10      1.802      0.078   -1.76e+10    3.25e+11
Federal_rate       4.177e+09   1.44e+09      2.900      0.006    1.28e+09    7.07e+09
==============================================================================
Omnibus:                       21.376   Durbin-Watson:                   1.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               49.659
Skew:                           1.052   Prob(JB):                     1.65e-11
Kurtosis:                       7.106   Cond. No.                     1.46e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.46e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [91]:
# Regress mean market cap on GDP growth and controlling variables for all sectors

import statsmodels.regression.linear_model as sm

list_of_sectors = ["mean_communication","mean_consumerdisc","mean_consumerstap","mean_energy","mean_financials","mean_healthcare","mean_industrials","mean_informationtech","mean_materials","mean_realestate","mean_utilities"]

for resp in list_of_sectors:
    formula = resp + " ~ Real_GDP + Unemployment_rate + Crude_price + Delta_Fed_Debt + Federal_rate"
    result = sm.OLS.from_formula(formula, data = data_factors_merged).fit()
    result.summary()

In [95]:
import statsmodels.formula.api as sm

result = sm.ols(formula="mean_communication ~ Real_GDP + Unemployment_rate + Crude_price + Delta_Fed_Debt + Federal_rate", data=data_factors_merged).fit()
print(result.params())


TypeError: 'Series' object is not callable